# Analyze Heart Disease with Amazon SageMaker XGBoost
---

In [9]:
# cell 00 .. install and setup dependent libraries and SDKs for Jupyter Notebook
%pip install sagemaker-experiments

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [50]:
# cell 01
import sagemaker

import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
session = sagemaker.Session()
s3_bucket = session.default_bucket()
s3_data_prefix = 'sagemaker/heartdisease/data/'
s3_model_prefix = 'sagemaker/heartdisease/xgboost'
s3_output_path = 's3://{}/{}/output'.format(s3_bucket, s3_model_prefix)
algorithm = 'xgboost'
trial_prefix = 'sm-heart-xgb-trial'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)
s3 = boto3.Session().resource('s3')

print(s3_bucket)

sagemaker-us-east-2-645411899653


In [51]:
# cell 02 ... ETL
s3_remote_path = s3_data_prefix + 'heart_failure_clinical_records_data-02-processed.csv'
sm_local_path = 'heart_failure_clinical_records_data-02-processed.csv'

# download file from remote to local
s3.Bucket(s3_bucket).download_file( s3_remote_path, sm_local_path)

In [52]:
# cell 03 ... preview input data frame ... 299 rows x 12 columns
import pandas as pd

data = pd.read_csv(sm_local_path)
pd.set_option('display.max_columns', 50)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 50)         # Keep the output on one page
data

,target_heart_failure,sex,smoking,diabetes,anaemia,high_blood_pressure,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium
0,1,1,0,0,0,1,1.192945,0.000166,-1.530560,1.681648e-02,0.490057,-1.504036
1,1,1,0,0,0,0,-0.491279,7.514640,-0.007077,7.535660e-09,-0.284552,-0.141976
2,1,1,1,0,0,0,0.350833,-0.449939,-1.530560,-1.038073e+00,-0.090900,-1.731046
3,1,1,0,0,1,0,-0.912335,-0.486071,-1.530560,-5.464741e-01,0.490057,0.085034
4,1,0,0,1,1,0,0.350833,-0.435486,-1.530560,6.517986e-01,1.264666,-4.682176
...,...,...,...,...,...,...,...,...,...,...,...,...
294,0,1,1,1,0,1,0.098199,-0.537688,-0.007077,-1.109765e+00,-0.284552,1.447094
295,0,0,0,0,0,0,-0.491279,1.278215,-0.007077,6.802472e-02,-0.187726,0.539054
296,0,0,0,1,0,0,-1.333392,1.525979,1.854958,4.902082e+00,-0.575031,0.312044
297,0,1,1,0,0,0,-1.333392,1.890398,-0.007077,-1.263389e+00,0.005926,0.766064


In [53]:
# cell 04 ... split data set into training and test subsets
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

In [54]:
# cell 05 .. publish training and test subsets of data to S3
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=False)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=s3_model_prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=s3_model_prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-2-645411899653/sagemaker/heartdisease/xgboost/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-2-645411899653/sagemaker/heartdisease/xgboost/test/test_data.csv


## Launching the SageMaker HyperParameter Tuning Job<a name="Launching"></a>

In [55]:
# cell 06 ... create parent experiment to associate with HyperParameterTuner tuning job

import time
from time import strftime

import smexperiments
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

create_date = strftime("%Y-%m-%d")
experiment_prefix = 'sm-heart-exp' 
experiment_name = 'sm-heart-exp-{}'.format(create_date)
trial_name = '{}-{}'.format(trial_prefix, create_date)

# experiment
try:
    experiment = Experiment.load(experiment_name = experiment_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        experiment = Experiment.create(experiment_name = experiment_name, 
                                       description = "SageMaker Heart Disease experiment", 
                                       tags = [{'Key': 'Experiment', 'Value': experiment_name}])
# trial

try:
    trial = Trial.load(trial_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        trial = Trial.create(experiment_name=experiment_name, trial_name=trial_name)
        
print(experiment_name)
print(trial_name)

sm-heart-exp-2021-12-08
sm-heart-xgb-trial-2021-12-08


In [56]:
# cell 07
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig

# get algo container [class]
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework=algorithm, version='latest')
# setup estimator for algo [tbt running instance of algo class]
ml_model = sagemaker.estimator.Estimator(                
                container,
                role, 
                instance_count=1, 
                instance_type='ml.m4.xlarge',
                output_path=s3_output_path,
                sagemaker_session=session,
                tags = [{'Key':'Experiment','Value':experiment_name},{'Key':'Trial','Value':trial_name}],
                debugger_hook_config=DebuggerHookConfig(
                    s3_output_path = s3_output_path,
                    collection_configs = [
                        CollectionConfig(name="metrics",parameters={"save_interval":"5"}),
                        CollectionConfig(name="predictions",parameters={"save_interval":"5"}),
                        CollectionConfig(name="feature_importance",parameters={"save_interval":"5"}),
                        CollectionConfig(name="average_shap", parameters={"save_interval":"5"})
                    ]
                )                  
)

# setup training and test/validation channels
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(s3_bucket, s3_model_prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test/'.format(s3_bucket, s3_model_prefix), content_type='csv')
print('Defined ML model estimator for {}'.format(trial_name))

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


Defined ML model estimator for sm-heart-xgb-trial-2021-12-08


In [57]:
#cell 08 ... HyperParameterTuner.Job.Run()

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

ml_model.set_hyperparameters( num_round = 100)

# define hyperparameter ranges
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html ... alpha, min_child_weight, eta, num_round
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 100),
                            'max_depth': IntegerParameter(1, 10)}

# define metric ... F1 = harmonic mean of precision and recall
objective_metric_name = 'validation:auc'

# define HyperParameterTuner job
ml_tuner = HyperparameterTuner(ml_model,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=100,
                            max_parallel_jobs=5,
                            base_tuning_job_name = trial_prefix,
                           tags = [{'Key':'Experiment','Value':experiment_name},{'Key':'Trial','Value':trial_name}])

print('Defined and started ML model HyperParameterTuner job for {}'.format(trial_prefix))

# job.Run()
ml_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: sm-heart-xgb-trial-211208-1511


Defined and running ML model HyperParameterTuner job for sm-heart-xgb-trial
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [59]:
# cell09 ... associate HyperParameterTuner Job instances with Trials in parent Experiment

import time
from datetime import timezone
from smexperiments.search_expression import Filter, Operator, SearchExpression

# get the most recently created tuning job

list_tuning_jobs_response = sm.list_hyper_parameter_tuning_jobs(
    SortBy="CreationTime", SortOrder="Descending", NameContains=trial_prefix
)
print(f'Found {len(list_tuning_jobs_response["HyperParameterTuningJobSummaries"])} tuning jobs.')

tuning_jobs = list_tuning_jobs_response["HyperParameterTuningJobSummaries"]
most_recently_created_tuning_job = tuning_jobs[0]

print(f"Found HyperParameterTuningJob {most_recently_created_tuning_job}")

creation_time = most_recently_created_tuning_job["CreationTime"]
creation_time = creation_time.astimezone(timezone.utc)
creation_time = creation_time.strftime("%Y-%m-%dT%H:%M:%SZ")

created_after_filter = Filter(
    name="CreationTime",
    operator=Operator.GREATER_THAN_OR_EQUAL,
    value=str(creation_time),
)
source_arn_filter = Filter(
    name="TrialComponentName", operator=Operator.CONTAINS, value=trial_prefix
)
source_type_filter = Filter(
    name="Source.SourceType", operator=Operator.EQUALS, value="SageMakerTrainingJob"
)

search_expression = SearchExpression(
    filters=[created_after_filter, source_arn_filter, source_type_filter]
)

# search for related training trials
trial_component_search_results = list(
    TrialComponent.search(search_expression=search_expression, sagemaker_boto_client=sm)
)
print(f"Found {len(trial_component_search_results)} trial components.")

# create trial
try:
    trial = Trial.load(trial_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        trial = Trial.create(experiment_name=experiment_name, trial_name=trial_name)

# associate trial components to trial
for tc in trial_component_search_results:
    print(f"Associating trial component {tc.trial_component_name} with trial {trial.trial_name}.")
    trial.add_trial_component(tc.trial_component_name)
    # sleep to avoid throttling
    time.sleep(1)

Found 2 tuning jobs.
Found HyperParameterTuningJob {'HyperParameterTuningJobName': 'sm-heart-xgb-trial-211208-1511', 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-2:645411899653:hyper-parameter-tuning-job/sm-heart-xgb-trial-211208-1511', 'HyperParameterTuningJobStatus': 'Completed', 'Strategy': 'Bayesian', 'CreationTime': datetime.datetime(2021, 12, 8, 15, 11, 34, 219000, tzinfo=tzlocal()), 'HyperParameterTuningEndTime': datetime.datetime(2021, 12, 8, 16, 28, 29, 367000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 12, 8, 16, 28, 29, 367000, tzinfo=tzlocal()), 'TrainingJobStatusCounters': {'Completed': 100, 'InProgress': 0, 'RetryableError': 0, 'NonRetryableError': 0, 'Stopped': 0}, 'ObjectiveStatusCounters': {'Succeeded': 100, 'Pending': 0, 'Failed': 0}, 'ResourceLimits': {'MaxNumberOfTrainingJobs': 100, 'MaxParallelTrainingJobs': 5}}
Found 100 trial components.
Associating trial component sm-heart-xgb-trial-211208-1511-100-38489f09-aws-training-job with t

In [65]:
# cell 10 ... select best training job per the eval metric
# HyperParameterTuner.model.select
ml_tuner.describe()
ml_best_estimator = ml_tuner.best_estimator()
# model name
ml_best_job = ml_tuner.best_training_job()

# log
print(ml_best_estimator)
print(ml_best_job)

# deploy
# HyperParameterTuner.model.deploy
ml_tuner_predictor = ml_tuner.deploy(model_name = ml_best_job, initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=sagemaker.serializers.CSVSerializer())


2021-12-08 16:04:00 Starting - Preparing the instances for training
2021-12-08 16:04:00 Downloading - Downloading input data
2021-12-08 16:04:00 Training - Training image download completed. Training in progress.
2021-12-08 16:04:00 Uploading - Uploading generated training model
2021-12-08 16:04:00 Completed - Training job completed
sm-heart-xgb-trial-211208-1511-066-6d527f8d

2021-12-08 16:04:00 Starting - Preparing the instances for training
2021-12-08 16:04:00 Downloading - Downloading input data
2021-12-08 16:04:00 Training - Training image download completed. Training in progress.
2021-12-08 16:04:00 Uploading - Uploading generated training model
2021-12-08 16:04:00 Completed - Training job completed

INFO:sagemaker:Creating model with name: sm-heart-xgb-trial-211208-1511-066-6d527f8d


INFO:sagemaker:Creating endpoint with name sm-heart-xgb-trial-211208-1511-066-6d527f8d


-------!

In [66]:
# cell 11 ... evaluate best model performance
test_features = test_data.drop(['target_heart_failure'], axis=1)
test_labels = test_data['target_heart_failure']

In [67]:
# cell 12 .. calculate predictions for XGBoost
import numpy as np
import json

def predict(predictor, data, rows=500, verbose=False):
    predictions = ''
    for row in data:
        if (verbose):
            print(row)
        prediction = predictor.predict(row, initial_args={"ContentType":"text/csv"})
        if (verbose):
            print(prediction)
        predictions = ','.join([predictions, prediction.decode('utf-8')])

    return np.fromstring(predictions[1:],sep=',')


In [68]:
# cell 13 ... analyze confusion matrix on the test data set
result = ml_tuner_predictor.predict([1,0,0,0,0,-0.491279276,7.514639529,-0.00707675,7.54E-09,-0.284552352,-0.141976151], initial_args={"ContentType":"text/csv"})
print(result)

# print (test_features)
predictions = predict(ml_tuner_predictor, test_features.to_numpy(), verbose=False)

# print predictions
print(predictions)

# calculate confusion matrix
pd.crosstab(index=test_labels, columns=np.round(predictions), rownames=["actual"],colnames=["predictions"])

b'0.3050850033760071'
[ 3.05085003e-01  8.95835161e-01  8.47165465e-01  2.80080974e-01
  3.73391271e-01  2.77712405e-01  7.08387852e-01  5.07210076e-01
  1.12329984e+00  1.54689878e-01  1.98707044e-01  3.00971568e-02
  4.66750652e-01 -3.24410200e-03  1.35328084e-01  3.27265263e-01
  8.50449204e-01  1.85523421e-01  4.49134052e-01  6.43275976e-01
  1.78216130e-01  2.05171466e-01  9.60416496e-02  4.96752292e-01
  5.79710305e-01  3.77840579e-01  6.00510359e-01  3.25509906e-01
  5.66852987e-02  1.98491186e-01  1.79987103e-01  4.65506971e-01
  4.58057165e-01  3.41871440e-01  6.88701868e-04  7.44679093e-01
  7.62424469e-02  7.67049074e-01  9.27446187e-02  2.02690065e-01
  2.05144316e-01  2.98188925e-01  8.32847357e-01  2.47427315e-01
  1.74433559e-01  1.50965631e-01  2.22236633e-01  1.50896460e-01
  3.13879251e-02  2.72049576e-01  1.74014390e-01  2.98213303e-01
  8.26743841e-01  9.59082842e-02  1.49143875e-01  1.82528198e-01
  9.72365141e-02  6.84426129e-01  2.72822380e-02 -1.81217790e-01]


predictions,0.0,1.0
actual,,
0,37,6
1,9,8


In [70]:
# cell 14 ... calculate classification metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

# round
np_predictions = np.round(predictions)

print('Precision = {}'.format(precision_score(test_labels, np_predictions, average='weighted', zero_division=1)))
print('Recall = {}'.format(recall_score(test_labels, np_predictions, average='weighted', zero_division=1)))
print('Accuracy = {}'.format(accuracy_score(test_labels, np_predictions)))
print('F1 = {}'.format(f1_score(test_labels, np_predictions, average='weighted', zero_division=1)))
print('AUC = {}'.format(roc_auc_score(test_labels, np_predictions, average='weighted')))

Precision = 0.7383540372670808
Recall = 0.75
Accuracy = 0.75
F1 = 0.742116708952519
AUC = 0.6655266757865937


In [46]:
# cell 15a ... Clarify ... prepare pre-training and post-training bias checks

from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=session)

bias_report_output_path = 's3://{}/{}/clarify-bias'.format(s3_bucket, s3_model_prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_data_s3_path,
                                      s3_output_path=bias_report_output_path,
                                      label='target_heart_failure',
                                      headers=train_data.columns.to_list(),
                                      dataset_type='text/csv')

model_config = clarify.ModelConfig(model_name=ml_best_job,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

bias_config = clarify.BiasConfig(label_values_or_threshold=[1],
                                facet_name='sex',
                                facet_values_or_threshold=[0],
                                group_name='age')

[]


IndexError: list index out of range

In [ ]:
# cell 15b ... Clarify .. explain inference results

shap_config = clarify.SHAPConfig(baseline=[test_features.iloc[0].values.tolist()],
                                 num_samples=30,
                                 agg_method='mean_abs',
                                 save_local_shap_values=True)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, s3_model_prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='target_heart_failure',
                                headers=train_data.columns.to_list(),
                                dataset_type='text/csv')

In [71]:
# test


2021-12-08 16:04:00 Starting - Preparing the instances for training
2021-12-08 16:04:00 Downloading - Downloading input data
2021-12-08 16:04:00 Training - Training image download completed. Training in progress.
2021-12-08 16:04:00 Uploading - Uploading generated training model
2021-12-08 16:04:00 Completed - Training job completed


In [ ]:
# cell zzz ... cleanup
# delete endpoint
# delete model
# delete s3 bucket